# Marine ML Benchmark: Demo Reproduction Notebook

This notebook demonstrates how to reproduce the key results from the Marine ML Benchmark study.

## Overview

This notebook will walk you through:
1. Loading and exploring the processed datasets
2. Training models on sample data
3. Evaluating model performance
4. Generating visualizations
5. Reproducing key findings

**Note**: This is a demonstration notebook. For full reproduction, use the provided shell scripts.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Machine learning libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# Set random seed for reproducibility
np.random.seed(42)

# Configure plotting
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("✅ Libraries imported successfully")

## 1. Data Exploration

Let's start by exploring the processed datasets to understand their characteristics.

In [ ]:
# Load dataset characteristics
data_dir = Path('../../data/processed')
tables_dir = Path('../../outputs/tables')

# Load dataset summary
if (tables_dir / 'final_table1_dataset_characteristics.csv').exists():
    dataset_chars = pd.read_csv(tables_dir / 'final_table1_dataset_characteristics.csv')
    print("📊 Dataset Characteristics:")
    print(dataset_chars.to_string(index=False))
else:
    print("⚠️  Dataset characteristics table not found. Run the full pipeline first.")

In [ ]:
# Explore a sample dataset
sample_dataset = 'cleaned_data'
sample_path = data_dir / sample_dataset / 'clean.csv'

if sample_path.exists():
    df_sample = pd.read_csv(sample_path)
    
    print(f"📈 Sample Dataset: {sample_dataset}")
    print(f"Shape: {df_sample.shape}")
    print(f"Columns: {list(df_sample.columns[:10])}{'...' if len(df_sample.columns) > 10 else ''}")
    print(f"\nFirst few rows:")
    print(df_sample.head())
    
    # Basic statistics
    print(f"\n📊 Basic Statistics:")
    print(df_sample.describe())
else:
    print(f"⚠️  Sample dataset {sample_dataset} not found.")
    print("Available datasets:")
    for dataset_dir in data_dir.glob('*/'):
        if (dataset_dir / 'clean.csv').exists():
            print(f"  - {dataset_dir.name}")

## 2. Model Training Demo

Let's demonstrate model training on a sample dataset.

In [ ]:
# Prepare data for modeling
if 'df_sample' in locals() and not df_sample.empty:
    # Identify target column (assuming it's the last numerical column or contains 'chla')
    numerical_cols = df_sample.select_dtypes(include=[np.number]).columns
    
    # Try to find chlorophyll-a related column
    target_candidates = [col for col in numerical_cols if 'chla' in col.lower() or 'chlorophyll' in col.lower()]
    
    if target_candidates:
        target_col = target_candidates[0]
    else:
        target_col = numerical_cols[-1]  # Use last numerical column
    
    print(f"🎯 Target variable: {target_col}")
    
    # Prepare features and target
    feature_cols = [col for col in numerical_cols if col != target_col]
    X = df_sample[feature_cols].fillna(df_sample[feature_cols].mean())
    y = df_sample[target_col].fillna(df_sample[target_col].mean())
    
    print(f"📊 Features: {len(feature_cols)} columns")
    print(f"📊 Samples: {len(X)} observations")
    print(f"📊 Target range: [{y.min():.4f}, {y.max():.4f}]")
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    print(f"\n✅ Data prepared for modeling")
    print(f"   Training samples: {len(X_train)}")
    print(f"   Test samples: {len(X_test)}")
else:
    print("⚠️  No sample data available for modeling demo")

In [ ]:
# Train multiple models
if 'X_train' in locals():
    models = {
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42, verbosity=0)
    }
    
    results = {}
    
    print("🤖 Training models...")
    
    for name, model in models.items():
        print(f"\n   Training {name}...")
        
        # Train model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        
        # Calculate metrics
        train_r2 = r2_score(y_train, y_pred_train)
        test_r2 = r2_score(y_test, y_pred_test)
        test_mae = mean_absolute_error(y_test, y_pred_test)
        test_rmse = mean_squared_error(y_test, y_pred_test, squared=False)
        
        results[name] = {
            'Train R²': train_r2,
            'Test R²': test_r2,
            'Test MAE': test_mae,
            'Test RMSE': test_rmse
        }
        
        print(f"     Train R²: {train_r2:.4f}")
        print(f"     Test R²: {test_r2:.4f}")
        print(f"     Test MAE: {test_mae:.4f}")
    
    # Display results
    results_df = pd.DataFrame(results).T
    print("\n📊 Model Performance Summary:")
    print(results_df.round(4))
else:
    print("⚠️  Cannot train models without prepared data")

## 3. Cross-Validation Demo

Demonstrate cross-validation for more robust performance estimation.

In [ ]:
# Cross-validation demonstration
if 'X' in locals() and 'y' in locals():
    print("🔄 Cross-Validation Analysis")
    
    # Use TimeSeriesSplit for demonstration (even if not time series)
    cv = TimeSeriesSplit(n_splits=5)
    
    cv_results = {}
    
    for name, model in models.items():
        print(f"\n   {name} Cross-Validation:")
        
        # Perform cross-validation
        cv_scores = cross_val_score(model, X, y, cv=cv, scoring='r2')
        
        cv_results[name] = {
            'Mean R²': cv_scores.mean(),
            'Std R²': cv_scores.std(),
            'Min R²': cv_scores.min(),
            'Max R²': cv_scores.max()
        }
        
        print(f"     Mean R²: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
        print(f"     Range: [{cv_scores.min():.4f}, {cv_scores.max():.4f}]")
    
    # Display CV results
    cv_results_df = pd.DataFrame(cv_results).T
    print("\n📊 Cross-Validation Summary:")
    print(cv_results_df.round(4))
else:
    print("⚠️  Cannot perform cross-validation without data")

## 4. Visualization Demo

Create some basic visualizations to understand model performance.

In [ ]:
# Visualization of results
if 'results_df' in locals():
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: Model performance comparison
    results_df[['Test R²', 'Test MAE']].plot(kind='bar', ax=axes[0])
    axes[0].set_title('Model Performance Comparison')
    axes[0].set_ylabel('Score')
    axes[0].legend()
    axes[0].tick_params(axis='x', rotation=45)
    
    # Plot 2: Prediction vs Actual (for best model)
    if 'models' in locals() and 'X_test' in locals():
        best_model_name = results_df['Test R²'].idxmax()
        best_model = models[best_model_name]
        y_pred_best = best_model.predict(X_test)
        
        axes[1].scatter(y_test, y_pred_best, alpha=0.6)
        axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        axes[1].set_xlabel('Actual Values')
        axes[1].set_ylabel('Predicted Values')
        axes[1].set_title(f'Predictions vs Actual ({best_model_name})')
        
        # Add R² to plot
        r2_best = r2_score(y_test, y_pred_best)
        axes[1].text(0.05, 0.95, f'R² = {r2_best:.4f}', 
                    transform=axes[1].transAxes, fontsize=12,
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    print(f"✅ Visualizations generated")
else:
    print("⚠️  No results available for visualization")

## 5. Load and Display Paper Results

Load and display the actual results from the full benchmark study.

In [ ]:
# Load paper results
print("📊 Loading Paper Results...")

# Load performance results
if (tables_dir / 'final_table2_model_performance.csv').exists():
    performance_results = pd.read_csv(tables_dir / 'final_table2_model_performance.csv')
    
    print("\n🏆 Model Performance Results (Sample):")
    print(performance_results.head(10))
    
    # Summary statistics
    print("\n📈 Performance Summary by Model:")
    model_summary = performance_results.groupby('Model')['R²'].agg(['mean', 'std', 'count']).round(4)
    print(model_summary)
    
else:
    print("⚠️  Performance results not found. Run the full pipeline first.")

# Load best performance results
if (tables_dir / 'final_table3_best_performance.csv').exists():
    best_results = pd.read_csv(tables_dir / 'final_table3_best_performance.csv')
    
    print("\n🥇 Best Model per Dataset:")
    print(best_results.to_string(index=False))
else:
    print("⚠️  Best performance results not found.")

In [ ]:
# Create summary visualization of paper results
if 'performance_results' in locals():
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Plot 1: Performance by model
    model_perf = performance_results.groupby('Model')['R²'].mean().sort_values(ascending=False)
    model_perf.plot(kind='bar', ax=axes[0,0], color='skyblue')
    axes[0,0].set_title('Average R² by Model')
    axes[0,0].set_ylabel('R² Score')
    axes[0,0].tick_params(axis='x', rotation=45)
    
    # Plot 2: Performance by dataset
    dataset_perf = performance_results.groupby('Dataset')['R²'].mean().sort_values(ascending=False)
    dataset_perf.plot(kind='bar', ax=axes[0,1], color='lightcoral')
    axes[0,1].set_title('Average R² by Dataset')
    axes[0,1].set_ylabel('R² Score')
    axes[0,1].tick_params(axis='x', rotation=45)
    
    # Plot 3: Performance distribution
    performance_results.boxplot(column='R²', by='Model', ax=axes[1,0])
    axes[1,0].set_title('R² Distribution by Model')
    axes[1,0].set_xlabel('Model')
    axes[1,0].tick_params(axis='x', rotation=45)
    
    # Plot 4: Model robustness (std vs mean)
    model_stats = performance_results.groupby('Model')['R²'].agg(['mean', 'std'])
    axes[1,1].scatter(model_stats['mean'], model_stats['std'])
    for model, (mean_r2, std_r2) in model_stats.iterrows():
        axes[1,1].annotate(model, (mean_r2, std_r2), xytext=(5, 5), 
                          textcoords='offset points', fontsize=10)
    axes[1,1].set_xlabel('Mean R²')
    axes[1,1].set_ylabel('Standard Deviation R²')
    axes[1,1].set_title('Model Robustness (Lower std = More robust)')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Paper results visualization completed")
else:
    print("⚠️  Cannot create paper results visualization without data")

## 6. Key Findings Summary

Summarize the key findings from the benchmark study.

In [ ]:
print("🔍 KEY FINDINGS SUMMARY")
print("=" * 50)

if 'performance_results' in locals():
    # Calculate key statistics
    model_stats = performance_results.groupby('Model')['R²'].agg(['mean', 'std', 'count'])
    best_model = model_stats['mean'].idxmax()
    best_mean_r2 = model_stats.loc[best_model, 'mean']
    best_std_r2 = model_stats.loc[best_model, 'std']
    
    dataset_stats = performance_results.groupby('Dataset')['R²'].agg(['mean', 'max', 'min'])
    easiest_dataset = dataset_stats['mean'].idxmax()
    hardest_dataset = dataset_stats['mean'].idxmin()
    
    print(f"🏆 BEST OVERALL MODEL: {best_model}")
    print(f"   Mean R²: {best_mean_r2:.4f} ± {best_std_r2:.4f}")
    print(f"   Evaluated on: {model_stats.loc[best_model, 'count']} datasets")
    
    print(f"\n📊 DATASET DIFFICULTY:")
    print(f"   Easiest: {easiest_dataset} (R² = {dataset_stats.loc[easiest_dataset, 'mean']:.4f})")
    print(f"   Hardest: {hardest_dataset} (R² = {dataset_stats.loc[hardest_dataset, 'mean']:.4f})")
    
    # Model ranking
    print(f"\n🥇 MODEL RANKING (by mean R²):")
    for i, (model, stats) in enumerate(model_stats.sort_values('mean', ascending=False).iterrows(), 1):
        print(f"   {i}. {model}: {stats['mean']:.4f} ± {stats['std']:.4f}")
    
    # Success rate analysis
    success_threshold = 0.5
    success_rate = (performance_results['R²'] > success_threshold).mean()
    print(f"\n✅ SUCCESS RATE (R² > {success_threshold}): {success_rate:.1%}")
    
    # Deep learning analysis
    dl_models = ['LSTM', 'TRANSFORMER']
    dl_results = performance_results[performance_results['Model'].isin(dl_models)]
    if not dl_results.empty:
        dl_datasets = dl_results['Dataset'].nunique()
        total_datasets = performance_results['Dataset'].nunique()
        print(f"\n🧠 DEEP LEARNING APPLICABILITY: {dl_datasets}/{total_datasets} datasets")
        print(f"   Limited by data structure requirements")
    
else:
    print("⚠️  Full results not available. Key findings from paper:")
    print("\n🏆 BEST OVERALL MODEL: XGBoost")
    print("   Mean R²: 0.823 ± 0.320")
    print("   Most robust across heterogeneous datasets")
    
    print("\n📊 DATASET DIFFICULTY:")
    print("   Easy (R² > 0.9): 6 datasets")
    print("   Medium (0.6 < R² < 0.9): 2 datasets")
    print("   Hard (R² < 0.3): 1 dataset (biotoxin)")
    
    print("\n🧠 DEEP LEARNING LIMITATION:")
    print("   Only applicable to 5/9 datasets")
    print("   Requires sufficient temporal structure")

print("\n💡 PRACTICAL IMPLICATIONS:")
print("   • Data quality more important than quantity")
print("   • XGBoost recommended for most oceanographic datasets")
print("   • Deep learning requires careful data structure assessment")
print("   • Cross-dataset validation essential for robustness")

print("\n" + "=" * 50)
print("✨ Demo completed! For full reproduction, run the pipeline scripts.")

## Next Steps

This notebook provided a demonstration of the key concepts and results. For complete reproduction:

1. **Quick Test**: Run `bash code/scripts/run_quick_test.sh` to verify installation
2. **Full Pipeline**: Run `bash code/scripts/run_full_pipeline.sh` to reproduce all results
3. **Custom Analysis**: Modify the code to explore specific aspects of interest
4. **Extension**: Add new datasets or models following the established framework

## References

- Paper: "Cross-Dataset Robustness of Machine Learning Models for Chlorophyll-a Prediction: A Comprehensive Benchmark Study"
- Repository: https://github.com/[username]/marine-ml-benchmark
- Documentation: See `docs/` directory for detailed methodology

---

*This notebook is part of the Marine ML Benchmark project. For questions or issues, please refer to the GitHub repository.*